In [1]:
import sys
import numpy as np
import math
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister, execute, BasicAer, IBMQ
from qiskit.visualization import plot_histogram
from qiskit.extensions import Initialize

In [2]:
#Arias
PIGEON_ALPHA_D =  "ATGCTGACCGACTCTGACAAGAAGCTGGTCCTGCAGGTGTGGGAGAAGGTGATCCGCCACCCAGACTGTGGAGCCGAGGCCCTGGAGAGGCTGTTCACCACCTACCCCCAGACCAAGACCTACTTCCCCCACTTCGACTTGCACCATGGCTCCGACCAGGTCCGCAACCACGGCAAGAAGGTGTTGGCCGCCTTGGGCAACGCTGTCAAGAGCCTGGGCAACCTCAGCCAAGCCCTGTCTGACCTCAGCGACCTGCATGCCTACAACCTGCGTGTCGACCCTGTCAACTTCAAGCTGCTGGCGCAGTGCTTCCACGTGGTGCTGGCCACACACCTGGGCAACGACTACACCCCGGAGGCACATGCTGCCTTCGACAAGTTCCTGTCGGCTGTGTGCACCGTGCTGGCCGAGAAGTACAGATAA"
DUCK_ALPHA_D =    "ATGCTGACCGCCGAGGACAAGAAGCTCATCGTGCAGGTGTGGGAGAAGGTGGCTGGCCACCAGGAGGAATTCGGAAGTGAAGCTCTGCAGAGGATGTTCCTCGCCTACCCCCAGACCAAGACCTACTTCCCCCACTTCGACCTGCATCCCGGCTCTGAACAGGTCCGTGGCCATGGCAAGAAAGTGGCGGCTGCCCTGGGCAATGCCGTGAAGAGCCTGGACAACCTCAGCCAGGCCCTGTCTGAGCTCAGCAACCTGCATGCCTACAACCTGCGTGTTGACCCTGTCAACTTCAAGCTGCTGGCACAGTGCTTCCAGGTGGTGCTGGCCGCACACCTGGGCAAAGACTACAGCCCCGAGATGCATGCTGCCTTTGACAAGTTCTTGTCCGCCGTGGCTGCCGTGCTGGCTGAAAAGTACAGATGA" 
CHICKEN_ALPHA_D = "ATGCTGACTGCCGAGGACAAGAAGCTCATCCAGCAGGCCTGGGAGAGGGCCGCTTCCCACCAGGAGGAGTTTGGAGCTGAGGCTCTGACTAGGATGTTCACCACCTATCCCCAGACCAAGACCTACTTCCCCCACTTCGACCTTTCGCCTGGCTCTGACCAGGTCCGTGGCCATGGCAAGAAGGTGTTGGGTGCCCTGGGCAACGCCGTGAAGAACGTGGACAACCTCAGCCAGGCCATGGCTGAGCTGAGCAACCTGCATGCCTACAACCTGCGTGTTGACCCCGTCAATTTCAAGCTGTTGTCGCAGTGCATCCAGGTGGTGCTGGCTGTACACATGGGCAAAGACTACACCCCTGAAGTGCATGCTGCCTTCGACAAGTTCCTGTCTGCCGTGTCTGCTGTGCTGGCTGAGAAGTACAGATAA"
#Tangarife
HORSE_ALPHA_2 = "ATGGTGCTGTCTGCCGCCGACAAGACCAACGTCAAGGCCGCCTGGAGTAAGGTTGGCGGCCACGCTGGCGAGTATGGCGCAGAGGCCCTAGAGAGGATGTTCCTGGGCTTCCCCACCACCAAGACCTACTTCCCCCACTTCGATCTGAGCCACGGCTCCGCCCAGGTCAAGGCCCACGGCCAGAAGGTGGGCGACGCGCTGACTCTCGCCGTGGGCCACCTGGACGACCTGCCTGGCGCCCTGTCGAATCTGAGCGACCTGCACGCACACAAGCTGCGCGTGGACCCCGTCAACTTCAAGCTCCTGAGTCATTGCCTGCTGTCCACCTTGGCCGTCCACCTCCCCAACGATTTCACCCCTGCCGTCCACGCCTCCCTGGACAAGTTCTTGAGCAGTGTGAGCACCGTGCTGACCTCCAAATACCGTTAA"
DUCK_ALPHA_D =  "ATGCTGACCGCCGAGGACAAGAAGCTCATCGTGCAGGTGTGGGAGAAGGTGGCTGGCCACCAGGAGGAATTCGGAAGTGAAGCTCTGCAGAGGATGTTCCTCGCCTACCCCCAGACCAAGACCTACTTCCCCCACTTCGACCTGCATCCCGGCTCTGAACAGGTCCGTGGCCATGGCAAGAAAGTGGCGGCTGCCCTGGGCAATGCCGTGAAGAGCCTGGACAACCTCAGCCAGGCCCTGTCTGAGCTCAGCAACCTGCATGCCTACAACCTGCGTGTTGACCCTGTCAACTTCAAGCTGCTGGCACAGTGCTTCCAGGTGGTGCTGGCCGCACACCTGGGCAAAGACTACAGCCCCGAGATGCATGCTGCCTTTGACAAGTTCTTGTCCGCCGTGGCTGCCGTGCTGGCTGAAAAGTACAGATGA"
GOAT_ALPHA_II = "ATGGTGCTGTCTGCCGCCGACAAGTCCAATGTCAAGGCCGCCTGGGGCAAGGTTGGCAGCAACGCTGGAGCTTATGGCGCAGAGGCTCTGGAGAGGATGTTCCTGAGCTTCCCCACCACCAAGACCTACTTCCCCCACTTCGACCTGAGCCACGGCTCGGCCCAGGTCAAGGGCCACGGCGAGAAGGTGGCCGCCGCGCTGACCAAAGCGGTGGGCCACCTGGACGACCTGCCCGGTACTCTGTCTGATCTGAGTGACCTGCACGCCCACAAGCTGCGTGTGGACCCGGTCAACTTTAAGCTTCTGAGCCACTCCCTGCTGGTGACCCTGGCCTGCCACCACCCCAGTGATTTCACCCCCGCGGTCCACGCCTCCCTGGACAAGTTCTTGGCCAACGTGAGCACCGTGCTGACCTCCAAATACCGTTAA" 

In [3]:
## crear la superpocisión cuántica de las secuencias genéticas
def encode_bitstring(bitstring, qr, cr, inverse=False):
    # El 2 representan la cantidad de qubit para el contenido de la secuencia
    n = math.ceil(math.log2(len(bitstring))) + 2 
    assert n > 3, "La longitud de la secuencia debe ser al menos 2"
        
    qc = QuantumCircuit(qr, cr)
    qc_init = QuantumCircuit(n)
    # Inicializar el vector deseado con ceros
    desired_vector = np.array([0.0 for i in range(2**n)])
    amplitude = np.sqrt(1.0/len(bitstring))
    
    for i, b in enumerate(bitstring):
        pos = i * 4
        # 00 => A , 01 => T, 10 => G, 11 => C
        if b == "T":
            pos += 1
        elif b == "G":
            pos += 2
        elif b == "C":
            pos += 3
        desired_vector[pos] = amplitude
    qc_init = Initialize(desired_vector)
    if not inverse:
        qc.append(qc_init, qr)
        qc.barrier(qr)
    else:
        qc.append(qc_init.gates_to_uncompute(), qr)
        qc.barrier(qr)
        for i in range(n):
            qc.measure(qr[i], cr[i])
    return qc

# Secuencias genéticas correspondientes a Arias

In [4]:
## Crear circuitos cuánticos para los estados cuanticos para las secuencias genéticas
ns = math.ceil(math.log2(len(PIGEON_ALPHA_D))) + 2
qr = QuantumRegister(ns)
cr = ClassicalRegister(ns)

qc_pigeon_alpha_d= encode_bitstring(PIGEON_ALPHA_D, qr, cr)
qc_duck_alpha_d = encode_bitstring(DUCK_ALPHA_D, qr, cr)
qc_chicken_alpha_d = encode_bitstring(CHICKEN_ALPHA_D, qr, cr)

circuitos = {"PIGEON_ALPHA_D": qc_pigeon_alpha_d, "DUCK_ALPHA_D": qc_duck_alpha_d, "CHICKEN_ALPHA_D": qc_chicken_alpha_d}

In [5]:
## Inversión de los circuitos cuánticos
inv_qc_pigeon_alpha_d = encode_bitstring(PIGEON_ALPHA_D, qr, cr, inverse=True)
inv_qc_duck_alpha_d = encode_bitstring(DUCK_ALPHA_D, qr, cr, inverse=True)
inv_qc_chicken_alpha_d = encode_bitstring(CHICKEN_ALPHA_D, qr, cr, inverse=True)

inv_circuitos = {"PIGEON_ALPHA_D": inv_qc_pigeon_alpha_d, "DUCK_ALPHA_D": inv_qc_duck_alpha_d, "CHICKEN_ALPHA_D": inv_qc_chicken_alpha_d}

In [6]:
## Comparar que tanta similitud tienen los códigos genéticos

key = "CHICKEN_ALPHA_D"

backend = BasicAer.get_backend("qasm_simulator")
shots=8192
combined_circs = {}
count = {}

most_similar, most_similar_score = "", -1.0

for other_key in inv_circuitos:
    if other_key == key:
        continue
    combined_circs[other_key] = circuitos[key] + inv_circuitos[other_key]
    job = execute(combined_circs[other_key], backend = backend, shots=shots)
    st = job.result().get_counts(combined_circs[other_key])
    if "0"*ns in st:
        sim_score = st["0"*ns]/shots
    else:
        sim_score = 0.0
    print("La similitud de la secuencia ", key, " y ", other_key, " es ", sim_score )
    if most_similar_score < sim_score:
        most_similar, most_similar_score = other_key, sim_score
        
print("RESPUESTA => ", key," es más similar a ", most_similar)


La similitud de la secuencia  CHICKEN_ALPHA_D  y  PIGEON_ALPHA_D  es  0.112548828125
La similitud de la secuencia  CHICKEN_ALPHA_D  y  DUCK_ALPHA_D  es  0.745849609375
RESPUESTA =>  CHICKEN_ALPHA_D  es más similar a  DUCK_ALPHA_D


# Secuencias genéticas correspondientes a Tangarife

In [ ]:
## Crear circuitos cuánticos para los estados cuanticos para las secuencias genéticas
ns = math.ceil(math.log2(len(HORSE_ALPHA_2))) + 2
qr = QuantumRegister(ns)
cr = ClassicalRegister(ns)

qc_horse_alpha_2= encode_bitstring(HORSE_ALPHA_2, qr, cr)
qc_duck_alpha_d = encode_bitstring(DUCK_ALPHA_D, qr, cr)
qc_goat_alpha_II = encode_bitstring(GOAT_ALPHA_II, qr, cr)

circuitos = {"HORSE_ALPHA_2": qc_horse_alpha_2, "DUCK_ALPHA_D": qc_duck_alpha_d, "GOAT_ALPHA_II": qc_goat_alpha_II}

In [ ]:
## Inversión de los circuitos cuánticos
inv_qc_horse_alpha_2= encode_bitstring(HORSE_ALPHA_2, qr, cr, inverse = True)
inv_qc_duck_alpha_d = encode_bitstring(DUCK_ALPHA_D, qr, cr, inverse = True)
inv_qc_goat_alpha_II = encode_bitstring(GOAT_ALPHA_II, qr, cr, inverse = True)

inv_circuitos = {"HORSE_ALPHA_2": inv_qc_horse_alpha_2, "DUCK_ALPHA_D": inv_qc_duck_alpha_d, "GOAT_ALPHA_II": inv_qc_goat_alpha_II}

In [ ]:
## Comparar que tanta similitud tienen los códigos genéticos

key = "GOAT_ALPHA_II"

backend = BasicAer.get_backend("qasm_simulator")
shots=8192
combined_circs = {}
count = {}

most_similar, most_similar_score = "", -1.0

for other_key in inv_circuitos:
    if other_key == key:
        continue
    combined_circs[other_key] = circuitos[key] + inv_circuitos[other_key]
    job = execute(combined_circs[other_key], backend = backend, shots=shots)
    st = job.result().get_counts(combined_circs[other_key])
    if "0"*ns in st:
        sim_score = st["0"*ns]/shots
    else:
        sim_score = 0.0
    print("La similitud de la secuencia ", key, " y ", other_key, " es ", sim_score )
    if most_similar_score < sim_score:
        most_similar, most_similar_score = other_key, sim_score
        
print("RESPUESTA => ", key," es más similar a ", most_similar)